Praca z bazą danych z poziomu kodu w języku Python z wykorzystaniem jedynie czystego języka SQL
może być problematyczna w przypadku większych projektów. Między innymi dlatego warto korzystać
z ułatwiających życie bibliotek. Jedną z takich bibliotek jest **SQL Alchemy**.

Na początek instalujemy bilbliotekę poleceniem:

```pip install sqlalchemy```

Teraz możemy ją zaimportować.

In [1]:
import sqlalchemy

Dla ułatwienia skorzystamy z przykładowej bazy danych sqlite, którą można znaleźć pod adresem
https://www.sqlitetutorial.net/sqlite-sample-database/

Tworzymy połączenie z bazą danych, pobieramy jej metadane i informacje o jednej z tabel.

In [44]:
engine = sqlalchemy.create_engine('sqlite:///chinook.db')
connection = engine.connect()
metadata = sqlalchemy.MetaData()
customers = sqlalchemy.Table('customers', metadata, autoload=True, autoload_with=engine)
tracks = sqlalchemy.Table('tracks', metadata, autoload=True, autoload_with=engine)
albums = sqlalchemy.Table('albums', metadata, autoload=True, autoload_with=engine)

Teraz sprawdźmy, jakie kolumny znajdują się w tabeli *customers*,

In [3]:
print(customers.columns.keys())

['CustomerId', 'FirstName', 'LastName', 'Company', 'Address', 'City', 'State', 'Country', 'PostalCode', 'Phone', 'Fax', 'Email', 'SupportRepId']


Możemy także wypisać bardziej szczegółowe dane na temat tej tabeli.

In [6]:
print(repr(metadata.tables['customers']))

Table('customers', MetaData(bind=None), Column('CustomerId', INTEGER(), table=<customers>, primary_key=True, nullable=False), Column('FirstName', NVARCHAR(length=40), table=<customers>, nullable=False), Column('LastName', NVARCHAR(length=20), table=<customers>, nullable=False), Column('Company', NVARCHAR(length=80), table=<customers>), Column('Address', NVARCHAR(length=70), table=<customers>), Column('City', NVARCHAR(length=40), table=<customers>), Column('State', NVARCHAR(length=40), table=<customers>), Column('Country', NVARCHAR(length=40), table=<customers>), Column('PostalCode', NVARCHAR(length=10), table=<customers>), Column('Phone', NVARCHAR(length=24), table=<customers>), Column('Fax', NVARCHAR(length=24), table=<customers>), Column('Email', NVARCHAR(length=60), table=<customers>, nullable=False), Column('SupportRepId', INTEGER(), ForeignKey('employees.EmployeeId'), table=<customers>), schema=None)


Oczywiście na bazie danych przede wszystkim chcemy wykonywać różnego rodzaju zapytania.

Żeby pobrać wszystkie dane z tabeli customers w języku SQL napisalibyśmy:

```SELECT * FROM Customers;```

Korzystając z *sqlalchemy* wystarczy utworzyć zapytanie:

In [7]:
query = sqlalchemy.select([customers])

Teraz wykonujemy je na bazie danych i wyświetlamy kilka pierwszych wierszy.

In [12]:
result_proxy = connection.execute(query)
result_set = result_proxy.fetchall()
print(result_set[:5])
result_proxy.close()

[(1, 'Luís', 'Gonçalves', 'Embraer - Empresa Brasileira de Aeronáutica S.A.', 'Av. Brigadeiro Faria Lima, 2170', 'São José dos Campos', 'SP', 'Brazil', '12227-000', '+55 (12) 3923-5555', '+55 (12) 3923-5566', 'luisg@embraer.com.br', 3), (2, 'Leonie', 'Köhler', None, 'Theodor-Heuss-Straße 34', 'Stuttgart', None, 'Germany', '70174', '+49 0711 2842222', None, 'leonekohler@surfeu.de', 5), (3, 'François', 'Tremblay', None, '1498 rue Bélanger', 'Montréal', 'QC', 'Canada', 'H2G 1A7', '+1 (514) 721-4711', None, 'ftremblay@gmail.com', 3), (4, 'Bjørn', 'Hansen', None, 'Ullevålsveien 14', 'Oslo', None, 'Norway', '0171', '+47 22 44 22 22', None, 'bjorn.hansen@yahoo.no', 4), (5, 'František', 'Wichterlová', 'JetBrains s.r.o.', 'Klanova 9/506', 'Prague', None, 'Czech Republic', '14700', '+420 2 4172 5555', '+420 2 4172 5555', 'frantisekw@jetbrains.com', 4)]


Gdy mamy do czynienia z dużymi zbiorami danych, pobranie całego wyniku zapytania naraz raczej nie będzie dobrym pomysłem.
By sobie poradzić z tym problemem, możemy dane pobierać partiami i na nich przeprowadzać operacje.

In [13]:
result_proxy = connection.execute(query)
while True:
    partial_res = result_proxy.fetchmany(20)
    if not partial_res:
        break

    print(partial_res[0])

result_proxy.close()

(1, 'Luís', 'Gonçalves', 'Embraer - Empresa Brasileira de Aeronáutica S.A.', 'Av. Brigadeiro Faria Lima, 2170', 'São José dos Campos', 'SP', 'Brazil', '12227-000', '+55 (12) 3923-5555', '+55 (12) 3923-5566', 'luisg@embraer.com.br', 3)
(21, 'Kathy', 'Chase', None, '801 W 4th Street', 'Reno', 'NV', 'USA', '89503', '+1 (775) 223-7665', None, 'kachase@hotmail.com', 5)
(41, 'Marc', 'Dubois', None, '11, Place Bellecour', 'Lyon', None, 'France', '69002', '+33 04 78 30 30 30', None, 'marc.dubois@hotmail.com', 5)


### Filtrowanie danych
Korzystanie z *SQL Alchemy* ułatwia także wykonywanie złożonych zapytać.
Przyjrzyjmy się kilku przykładom.

##### WHERE
```SELECT * FROM Customers WHERE Country = 'Brazil'```

In [ ]:
sqlalchemy.select([customers]).where(customers.columns.Country == "Brazli")

##### IN
```SELECT * FROM Customers WHERE Country IN ('Brazil', 'USA', 'France')```

In [ ]:
sqlalchemy.select([customers]).where(customers.columns.Country.in_(["Brazil", "USA", "France"]))

##### AND, OR, NOT
```SELECT * FROM Customers WHERE Country = 'Brazil' AND NOT FirstName = 'Luis'```

In [ ]:
sqlalchemy.select([customers]).where(sqlalchemy.and_(customers.columns.Country=="Brazil", customers.columns.FirstName!="Luis"))

##### ORDER BY
```SELECT * FROM Customers ORDER BY LastName DESC, FirstName```

In [ ]:
sqlalchemy.select([customers]).order_by(sqlalchemy.desc(customers.columns.LastName),customers.columns.FirstName)

##### Funkcje
```SELECT Count(*) FROM Customers WHERE Country = 'Brazil'```

In [22]:
sqlalchemy.select([sqlalchemy.func.count()]).where(customers.columns.Country == "Brazli")

<sqlalchemy.sql.selectable.Select at 0x16051a8; Select object>

##### GROUP BY
```SELECT SUM(UnitPrice) as PriceSum, AlbumId FROM Tracks GROUP BY AlbumId

In [23]:
sqlalchemy.select([sqlalchemy.func.sum(tracks.columns.UnitPrice).label("PriceSum"),tracks.columns.AlbumId]).group_by(tracks.columns.AlbumId)

<sqlalchemy.sql.selectable.Select at 0x8d66778; Select object>

##### DISTINCT
```SELECT DISTINCT Country FROM Customers```

In [ ]:
sqlalchemy.select([customers.columns.Country.distinct])

##### JOIN
Gdy pobieramy dane z wielu tabel mamy dwie opcje: wykorzystać istniejące relacje,
lub manualnie połączyć ze sobą tabele.

In [25]:
sqlalchemy.select([tracks.columns.Name, albums.columns.Title])

query = sqlalchemy.select([tracks, albums])
query = query.select_from(tracks.join(albums, tracks.columns.AlbumId == albums.columns.AlbumId))

##### Tworzenie tabel i dodawanie nowych danych

In [45]:
temp = sqlalchemy.Table("temp", metadata,
                        sqlalchemy.Column("Id", sqlalchemy.Integer()),
                        sqlalchemy.Column("name", sqlalchemy.String(255), nullable=False),
                        sqlalchemy.Column("value", sqlalchemy.Float())
                        )

metadata.create_all(engine)

query = sqlalchemy.insert(temp).values(Id=1, name="first", value=29.99)
connection.execute(query)

query = sqlalchemy.insert(temp)
val_list = [{'Id':2, 'name':'second', 'value':10.99}, {'Id':3, 'name':'third', 'value': 1.99}]
connection.execute(query, val_list)

##### Usuwanie tabeli

In [39]:
temp.drop(engine)

By ułatwić sobie pracę z danymi możemy skorzystać z biblioteki pandas.

In [48]:
import pandas
query = sqlalchemy.select([customers])
result_proxy = connection.execute(query)
result_set = result_proxy.fetchall()
result_proxy.close()
data_frame = pandas.DataFrame(result_set)
data_frame.columns = result_set[0].keys()
data_frame.head(5)

,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
2,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
3,4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
4,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4
